### Inicio

In [1]:
from datetime import date
import os
os.environ["AWS_PROFILE"] = "men"
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [2]:
params = {
    "class_name":"Inteligencia Artificial",
    "target_audience_edu_level":"preescolar",
    "grade":"no aplica"
}

### Retriever

In [3]:
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever

In [4]:
kb_context_id = "BC3JS4SRQG"
kb_result_id = "MQPBML7IQ2"

# Base knowledge
retriever_context = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_context_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

retriever_result = AmazonKnowledgeBasesRetriever(
    knowledge_base_id = kb_result_id,
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            'overrideSearchType': "SEMANTIC" # optional
        }
    },
)

In [5]:
def get_retriver_docs(docs):    
    context = "\n".join([doc.page_content for doc in docs])  # Iteramos de forma síncrona si 'docs' es una lista
    return context

In [6]:
async def get_retriver_context(params):
    docs = await retriever_context.ainvoke(params)
    return get_retriver_docs(docs)

In [7]:
async def get_retriver_result(params):
    docs = await retriever_result.ainvoke(params)
    return get_retriver_docs(docs)

### Prompts

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
def get_relevant_info():
    prompt_content = """
    Analiza si el tema "{class_name}" es adecuado para una clase educativa. Evalúa si cumple con las siguientes condiciones:
    1. Es un tema educativo que se puede enseñar en un entorno académico.
    2. No es ofensivo, vulgar, ni inapropiado para un entorno educativo.
    3. Tiene suficiente contenido o contexto educativo para ser desarrollado en una clase.
    4. Si el tema es el nombre de una persona:
       - Responde "NO" si la persona no es conocida por su relevancia educativa o histórica.
       - Responde "SI" únicamente si la persona es reconocida por una contribución educativa, científica, histórica, o cultural significativa.

    Responde únicamente con un **SI** si el tema es adecuado para una clase educativa o un **NO** si no lo es. No agregues ninguna explicación ni comentario adicional.
    """
    
    return PromptTemplate(
        input_variables=["class_name"],
        template=prompt_content,
    )



In [10]:
def get_base_prompt():
    with open("get_base_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()

    return PromptTemplate(
        input_variables=["target_audience_edu_level", "class_name", "grade", "date", "context", "learning_result"],
        template=prompt_content
    )

In [11]:
def get_key_concepts():
    with open("get_key_concepts.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["target_audience_edu_level", "class_name", "grade", "context"],
            template=prompt_content
        )

In [12]:
def get_activities_prompt():
    with open("get_activities_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["target_audience_edu_level", "class_name", "grade", "context", "dba"],
            template=prompt_content
        )

In [13]:
def get_exam_prompt():
    with open("get_exam_prompt.sty", "r", encoding="utf-8") as file:
        prompt_content = file.read()
    
    return PromptTemplate(
            input_variables=["prompt"],
            template=prompt_content
        )

### Definir el LLM


In [14]:
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock, ChatBedrockConverse


bedrock_model = "anthropic.claude-3-sonnet-20240229-v1:0"


llm_instance = ChatBedrockConverse(
    model=bedrock_model,
    temperature=0.02,
    max_tokens=None
)


### Objetos necesarios para definir la cadena y poder invocarla

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [16]:

def invoke_chain(prompt_template, params):
    
    # Create the LLM chain with prompt and parser
    chain = prompt_template | llm_instance | parser
    return chain.invoke(params)

def validate_input(class_name):

    prompt_content = """ consideras que se puede crear una clase formativa con este tema {class_name}, de no tener suficiente informacion o de que el tema no tenga que ver con aspectos educativos dame un NO por respuesta, de lo contrario dame un SI, delimitate a responder con un SI o un NO """

    appropiate_prompt = PromptTemplate(
            input_variables=["class_name"],
            template=prompt_content
        )

    return invoke_chain(appropiate_prompt, {"class_name": class_name})




In [17]:
relevance = get_relevant_info() | llm_instance | parser
relevance_params = {
    "class_name": params["class_name"]
}

relevante = relevance.invoke(relevance_params)



### Proceso Parte 1 :  Resultados de Aprendizaje

In [18]:
if  validate_input(params["class_name"]) == "NO" or relevante == 'NO':

  
  print("El tema que estás buscando no corresponde a un tema formativo, o actualmente no contamos con suficiente información para generar una clase sobre este asunto.")

else:
  prompt_template = get_base_prompt()
  classes = params["class_name"]
  context_query = f"{params['class_name']}, para grados {params['grade']}"
  context = get_retriver_context(context_query)

  dba_query = f"De los derechos basicos de aprendizaje dame la metodologia de enseñanza para la tematica de ({classes}) dirigida a estudiantes del grado {params['grade']}"
  dba = get_retriver_result(dba_query)

  learning_result_query = f"Dame información de los Estándares Básicos de Competencias y Orientaciones pedagógicas para la clase ({classes}) dirigida a estudiantes del grado {params['grade']}"
  learning_result = get_retriver_result(learning_result_query)

  chain = prompt_template | llm_instance | parser

  new_params = {
            "class_name": params["class_name"],
                "target_audience_edu_level":params["target_audience_edu_level"],
                "grade": params["grade"],
                "context": context,
                "dba": dba,
                "learning_result": learning_result,
                "date":  date.today()
            } 
  
  result_1 = chain.invoke(new_params)




In [19]:
print(result_1)

# ¡Explorando la Inteligencia Artificial!

Fecha de creación: 2025-01-17

Nivel educativo: Preescolar

Resultados de aprendizaje:

1. Identificar ejemplos de Inteligencia Artificial en su entorno cotidiano.
2. Desarrollar habilidades de observación y curiosidad sobre la tecnología.
3. Fomentar la creatividad y el pensamiento crítico a través de actividades lúdicas relacionadas con la Inteligencia Artificial.


### Proceso parte 2 : Conceptos clave

In [20]:
key_template = get_key_concepts()

chain_key_concepts = key_template | llm_instance | parser
key_params = {
    "class_name": params["class_name"],
    "target_audience_edu_level":params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context
}

result_2 = chain_key_concepts.invoke(key_params)

In [21]:
print(result_2)

## Conceptos clave:

- **Algoritmo**: Conjunto de instrucciones o reglas que se siguen para resolver un problema o realizar una tarea.
- **Aprendizaje automático**: Capacidad de los sistemas informáticos para aprender y mejorar a partir de la experiencia, sin ser programados explícitamente.
- **Datos**: Información que se utiliza para entrenar y alimentar los sistemas de inteligencia artificial.
- **Inteligencia artificial**: Rama de la informática que se encarga del desarrollo de sistemas capaces de realizar tareas que normalmente requieren inteligencia humana.
- **Máquina**: Dispositivo o sistema que realiza tareas de manera automática.
- **Reconocimiento de patrones**: Capacidad de identificar y clasificar patrones en los datos.
- **Robótica**: Rama de la inteligencia artificial que se enfoca en el diseño y construcción de robots.


### Proceso parte 3 : Actividades de aprendizaje

In [22]:
activities_template = get_activities_prompt()

chain_activities = activities_template | llm_instance | parser
activities_params = {
    "class_name": params["class_name"],
    "target_audience_edu_level": params["target_audience_edu_level"],
    "grade": params["grade"],
    "context": context,
    "dba": dba
}

result_3 = chain_activities.invoke(activities_params)

In [23]:
print(result_3)

## Metodologías

- Aprendizaje basado en juegos y actividades lúdicas, aprovechando el interés natural de los niños por el juego.
- Enfoque multisensorial, utilizando recursos visuales, auditivos y táctiles para facilitar la comprensión.
- Aprendizaje colaborativo, fomentando el trabajo en equipo y la interacción social.

## Actividades de aprendizaje

1. Introducción a la Inteligencia Artificial a través de cuentos y personajes animados.
2. Exploración de ejemplos cotidianos de IA, como asistentes virtuales o juguetes inteligentes.
3. Actividades de reconocimiento de patrones y categorización de objetos.
4. Juegos de resolución de problemas sencillos utilizando algoritmos básicos.
5. Creación de robots sencillos con materiales reciclados y programación básica.

## Actividades prácticas

- Construir robots sencillos con materiales reciclados y programarlos para realizar tareas simples.
- Juegos de reconocimiento de patrones y categorización de objetos.
- Actividades de programación bás

### Proceso parte 4 : Actividad evaluativa

In [24]:
exam_template = get_exam_prompt()
chain_exam = exam_template | llm_instance | parser
examn_params = {
    "prompt": result_1 + '\n\n' + result_2 + '\n\n' + result_3 
}

result_4 = chain_exam.invoke(examn_params)

In [25]:
print(result_4)

# Examen de Inteligencia Artificial

1. ¿Cuál de estas opciones es un ejemplo de Inteligencia Artificial en tu vida diaria?

A) Un robot de juguete
B) Un lápiz
C) Una pelota
D) Un libro

2. ¿Qué es un algoritmo?

A) Un conjunto de instrucciones para resolver un problema
B) Un tipo de máquina
C) Un juguete inteligente
D) Un robot

3. ¿Qué necesitan los sistemas de Inteligencia Artificial para aprender?

A) Libros
B) Datos
C) Lápices
D) Pelotas

4. ¿Qué es una máquina?

A) Un robot
B) Un dispositivo que realiza tareas automáticamente
C) Un algoritmo
D) Un juguete

5. ¿Qué es el reconocimiento de patrones?

A) Identificar y clasificar patrones en los datos
B) Construir robots
C) Programar juguetes
D) Leer libros

6. ¿Qué rama de la Inteligencia Artificial se enfoca en el diseño y construcción de robots?

A) Aprendizaje automático
B) Robótica
C) Algoritmos
D) Reconocimiento de patrones

7. ¿Cuál de estas actividades te ayuda a aprender sobre Inteligencia Artificial?

A) Jugar con robots
B)

In [26]:
def get_coherence_prompt():
    prompt_content = """
    Revisa que los temas presentados en la siguiente clase:

    "$result_1$
    {result_1}

    $result_2$
    {result_2}

    $result_3$
    {result_3}

    $result_4$
    {result_4}"

    sean coherentes entre sí y con el siguiente Nivel educativo: {target_audience_edu_level}, en caso de que lo veas coherente dejalo igual y en caso de que sean muy avanzados para el nivel educativo o que por el contrario sean muy basicos, ajustalo (por ejemplo si el nivel educativo es Preescolar cambialo para explicarselo a un niño de 4 años) y devuelve la respuesta en formato markdown igual al que tenía inicialmente pero asegurate de incluir los $result_x$ donde estaban. 
    No agregues la palabra markdown en la respuesta.
    Evita hacer cambios innecesarios, solo ajustalo si ver que en verdad no hay coherencia (ten un umbral bajo).
    """
    
    return PromptTemplate(
        input_variables=[
            "result_1",
            "result_2",
            "result_3",
            "result_4",
            "target_audience_edu_level"
        ],
        template=prompt_content,
    )


In [27]:
coherence = get_coherence_prompt() | llm_instance | parser
coherence_params = {
    "result_1": result_1,
    "result_2": result_2,
    "result_3": result_3,
    "result_4": result_4,
    "target_audience_edu_level": params["target_audience_edu_level"]
}

result_5 = coherence.invoke(coherence_params)

In [28]:
def split_results(full_text):
    final_results = {}
    sections = full_text.split("$result_")

    for section in sections:
        if section.strip(): 
            section_number, content = section.split("$", 1)
            key = f"final_result_{section_number.strip()}"
            final_results[key] = content.strip()

    return final_results

results = split_results(result_5)

final_result_1 = results.get("final_result_1", "")
final_result_2 = results.get("final_result_2", "")
final_result_3 = results.get("final_result_3", "")
final_result_4 = results.get("final_result_4", "")

In [ ]:
print(result_1)
print(result_2)
print(result_3)
print(result_4)

# ¡Explorando la Inteligencia Artificial!

Fecha de creación: 2025-01-17

Nivel educativo: Preescolar

Resultados de aprendizaje:

1. Identificar ejemplos de Inteligencia Artificial en su entorno cotidiano.
2. Desarrollar habilidades de observación y curiosidad sobre la tecnología.
3. Fomentar la creatividad y el pensamiento crítico a través de actividades lúdicas relacionadas con la Inteligencia Artificial.
## Conceptos clave:

- **Algoritmo**: Conjunto de instrucciones o reglas que se siguen para resolver un problema o realizar una tarea.
- **Aprendizaje automático**: Capacidad de los sistemas informáticos para aprender y mejorar a partir de la experiencia, sin ser programados explícitamente.
- **Datos**: Información que se utiliza para entrenar y alimentar los sistemas de inteligencia artificial.
- **Inteligencia artificial**: Rama de la informática que se encarga del desarrollo de sistemas capaces de realizar tareas que normalmente requieren inteligencia humana.
- **Máquina**: Disp

In [30]:
print(final_result_1)
print(final_result_2)
print(final_result_3)
print(final_result_4)

# ¡Explorando la Inteligencia Artificial!

Fecha de creación: 2025-01-17

Nivel educativo: Preescolar

Resultados de aprendizaje:

1. Identificar ejemplos de Inteligencia Artificial en su entorno cotidiano.
2. Desarrollar habilidades de observación y curiosidad sobre la tecnología.
3. Fomentar la creatividad y el pensamiento crítico a través de actividades lúdicas relacionadas con la Inteligencia Artificial.
## Conceptos clave:

- **Algoritmo**: Conjunto de instrucciones o pasos que se siguen para resolver un problema o realizar una tarea.
- **Aprendizaje automático**: Capacidad de las computadoras para aprender y mejorar a partir de la experiencia.
- **Datos**: Información que se utiliza para enseñar a las computadoras.
- **Inteligencia artificial**: Capacidad de las computadoras para realizar tareas que normalmente requieren inteligencia humana.
- **Máquina**: Dispositivo o sistema que realiza tareas de manera automática.
- **Reconocimiento de patrones**: Capacidad de identificar y c